In [ ]:
import pandas as pd
import numpy as np
import random
import os
from faker import Faker



In [ ]:
os.makedirs ("bonse", exist_ok=True)
faker = Faker()

In [ ]:
def gen_dim_alumno(n=30):
  alumno = []
  for i in range(1,n+1):
    alumno.append({
        'alumno_id': i,
        'nombre': faker.name(),
        'grado': random.choice(['1ro', '2do', '3ro']),
        'grupo': random.choice(['A', 'B', 'C'])
    })
  df = pd.DataFrame(alumno)
  df.to_parquet("bonse/alumno.parquet", index=False)
  return df

In [ ]:
def gen_dim_materia():
  materias = ["Matematicas", "Español", "Ciencia","Historia","Ingles"]
  df = pd.DataFrame ({
      'materia_id': range(1, len (materias)+1),
       'nombre_materia': materias
  })
  df.to_parquet("bonse/materia.parquet", index=False)
  return df

In [ ]:
def gen_dim_fecha(inicio='2024-01-01', fin='2024-03-3'):
  fechas = pd.date_range(start=inicio, end=fin, freq='D')
  df = pd.DataFrame({
      'fecha_id': range(1, len(fechas)+1),
      'fecha': fechas,
      'dia': fechas.day,
      'mes': fechas.month,
      'año': fechas.year,
      'dia_semana': fechas.day_name(),
      'es_laboral': fechas.weekday <5
  })
  df = df[df['es_laboral']]
  df.to_parquet("bonse/fecha.parquet", index=False)
  return df

In [ ]:
def gen_tabla_asistencia(df_fecha, df_alumno, df_materia, n=1000):
  asistencias = []
  for i in range(n):
    alumno = df_alumno.sample().iloc[0]
    materia = df_materia.sample().iloc[0]
    fecha = df_fecha.sample().iloc[0]
    presente = random.choice([True, False])
    asistencias.append({
        'registro_id': i+1,
        'fecha_id': fecha['fecha_id'],
        'alumno_id': alumno['alumno_id'],
        'materia_id': materia['materia_id'],
        'presente': presente
        })
  df = pd.DataFrame(asistencias)
  df.to_parquet("bonse/asistencia.parquet", index=False)
  return df


In [ ]:
df_alumno = gen_dim_alumno()
df_materia = gen_dim_materia()
df_fecha = gen_dim_fecha()
df_asistencia = gen_tabla_asistencia(df_fecha, df_alumno, df_materia)

In [ ]:
df_alumno = pd.read_parquet("bonse/alumno.parquet")
df_materia = pd.read_parquet("bonse/materia.parquet")
df_fecha = pd.read_parquet("bonse/fecha.parquet")
df_asistencia = pd.read_parquet("bonse/asistencia.parquet")

In [ ]:
df_join = df_asistencia.merge(df_fecha, on='fecha_id')
asistencia_por_mes = df_join.groupby('mes')['presente'].sum().reset_index()
display(asistencia_por_mes)

,mes,presente
0,1,245
1,2,243
2,3,16


from matplotlib import pyplot as plt
asistencia_por_mes['mes'].plot(kind='hist', bins=20, title='mes')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
asistencia_por_mes['presente'].plot(kind='hist', bins=20, title='presente')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
asistencia_por_mes.plot(kind='scatter', x='mes', y='presente', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['mes']
  ys = series['presente']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = asistencia_por_mes.sort_values('mes', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('mes')
_ = plt.ylabel('presente')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['mes']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'mes'}, axis=1)
              .sort_values('mes', ascending=True))
  xs = counted['mes']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = asistencia_por_mes.sort_values('mes', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('mes')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
asistencia_por_mes['mes'].plot(kind='line', figsize=(8, 4), title='mes')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
asistencia_por_mes['presente'].plot(kind='line', figsize=(8, 4), title='presente')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
os.makedirs("gold", exist_ok = True)


In [ ]:

df_join = df_asistencia.merge(df_fecha, on = 'fecha_id')
asistencia_por_mes = df_join.groupby('mes')['presente'].sum().reset_index()
asistencia_por_mes.to_parquet("gold/asistencia_por_mes.parquet", index = False)
display(asistencia_por_mes)

,mes,presente
0,1,245
1,2,243
2,3,16


from matplotlib import pyplot as plt
asistencia_por_mes['mes'].plot(kind='hist', bins=20, title='mes')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
asistencia_por_mes['presente'].plot(kind='hist', bins=20, title='presente')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
asistencia_por_mes.plot(kind='scatter', x='mes', y='presente', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['mes']
  ys = series['presente']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = asistencia_por_mes.sort_values('mes', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('mes')
_ = plt.ylabel('presente')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['mes']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'mes'}, axis=1)
              .sort_values('mes', ascending=True))
  xs = counted['mes']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = asistencia_por_mes.sort_values('mes', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('mes')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
asistencia_por_mes['mes'].plot(kind='line', figsize=(8, 4), title='mes')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
asistencia_por_mes['presente'].plot(kind='line', figsize=(8, 4), title='presente')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
df_join_alumno = df_asistencia.merge(df_alumno, on='alumno_id')
asistencia_por_grupo = df_join_alumno.groupby('grupo')['presente'].mean().reset_index()
asistencia_por_grupo.to_parquet("gold/asistencia_por_grupo.parquet", index=False)
display(asistencia_por_grupo)

,grupo,presente
0,A,0.523438
1,B,0.503623
2,C,0.493590


from matplotlib import pyplot as plt
asistencia_por_grupo['presente'].plot(kind='hist', bins=20, title='presente')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
asistencia_por_grupo.groupby('grupo').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
asistencia_por_grupo['presente'].plot(kind='line', figsize=(8, 4), title='presente')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(asistencia_por_grupo['grupo'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(asistencia_por_grupo, x='presente', y='grupo', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
df_join_materia = df_asistencia.merge(df_materia, on='materia_id')
inasistencias = df_join_materia[df_join_materia['presente'] == False].groupby('nombre_materia').size().reset_index(name='total_inasistencias')
inasistencias.to_parquet("gold/inasistencias_por_materia.parquet", index=False)
display(inasistencias)

,nombre_materia,total_inasistencias
0,Ciencia,82
1,Español,112
2,Historia,92
3,Ingles,105
4,Matematicas,105


from matplotlib import pyplot as plt
inasistencias['total_inasistencias'].plot(kind='hist', bins=20, title='total_inasistencias')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
inasistencias.groupby('nombre_materia').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
inasistencias['total_inasistencias'].plot(kind='line', figsize=(8, 4), title='total_inasistencias')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(inasistencias['nombre_materia'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(inasistencias, x='total_inasistencias', y='nombre_materia', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
df_join_fecha = df_asistencia.merge(df_fecha, on='fecha_id')
asistencia_por_mes = df_join_fecha.groupby('mes')['presente'].sum().reset_index()
display(asistencia_por_mes)
asistencia_por_mes.to_parquet("gold/asistencia_por_mes.parquet",index=False)

,mes,presente
0,1,245
1,2,243
2,3,16


from matplotlib import pyplot as plt
asistencia_por_mes['mes'].plot(kind='hist', bins=20, title='mes')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
asistencia_por_mes['presente'].plot(kind='hist', bins=20, title='presente')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
asistencia_por_mes.plot(kind='scatter', x='mes', y='presente', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['mes']
  ys = series['presente']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = asistencia_por_mes.sort_values('mes', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('mes')
_ = plt.ylabel('presente')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['mes']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'mes'}, axis=1)
              .sort_values('mes', ascending=True))
  xs = counted['mes']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = asistencia_por_mes.sort_values('mes', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('mes')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
asistencia_por_mes['mes'].plot(kind='line', figsize=(8, 4), title='mes')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
asistencia_por_mes['presente'].plot(kind='line', figsize=(8, 4), title='presente')
plt.gca().spines[['top', 'right']].set_visible(False)